# MDA project: predicting crowdedness in Leuven through noise and weather data

### Importing packages

In [11]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import shap

### Loading data and adding time-related features

In [5]:
df = pd.read_csv("...", delimiter=";")

### Exploratory data analyses

### Pipelines to fit all models (with cross-validation and hyperparameter tuning)

In [13]:
#pipeline to create feature matrix with delayed noise and weather data, as well as target vector.

def delay_noise_weather(df):
    # Select columns for df_noisedelay
    df_noisedelay = df[['object_id', 'result_timestamp', 'laeq']]

    # Add 6 hours to result_timestamp
    df_noisedelay['result_timestamp'] += pd.DateOffset(hours=6)

    # Create key column
    df_noisedelay['key'] = df_noisedelay['object_id'].astype(str) + df_noisedelay['result_timestamp'].astype(str)

    # Drop unnecessary columns
    df_noisedelay = df_noisedelay.drop(['laeq', 'object_id', 'result_timestamp'], axis=1)

    # Select columns for df_weatherdelay
    df_weatherdelay = df[['object_id', 'result_timestamp', 'LC_HUMIDITY', 'LC_DWPTEMP', 'LC_n', 'LC_RAD', 'LC_RAININ',
                          'LC_DAILYRAIN', 'LC_WINDDIR', 'LC_WINDSPEED', 'LC_RAD60', 'LC_TEMP_QCL0']]

    # Add 6 hours to result_timestamp
    df_weatherdelay['result_timestamp'] += pd.DateOffset(hours=6)

    # Create key column
    df_weatherdelay['key'] = df_weatherdelay['object_id'].astype(str) + df_weatherdelay['result_timestamp'].astype(str)

    # Drop unnecessary columns
    df_weatherdelay = df_weatherdelay.drop(['object_id', 'result_timestamp'], axis=1)

    # Drop weather-related columns from original df
    df = df.drop(['LC_HUMIDITY', 'LC_DWPTEMP', 'LC_n', 'LC_RAD', 'LC_RAININ',
                  'LC_DAILYRAIN', 'LC_WINDDIR', 'LC_WINDSPEED', 'LC_RAD60', 'LC_TEMP_QCL0'], axis=1)

    # Merge df with df_noisedelay and df_weatherdelay on 'key'
    merged_df = pd.merge(df, df_noisedelay, on='key').merge(df_weatherdelay, on='key')

    # Delete observations with missing values
    merged_df = merged_df.dropna()

    # Create feature matrix X and target vector y
    X = merged_df.drop(['result_timestamp', 'laeq'], axis=1)
    y = merged_df['laeq']

    # Return feature matrix X and target vector y
    return X, y

#Apply to df
X, y = delay_noise_weather(df)

TypeError: unsupported operand type(s) for +: 'DateOffset' and 'str'

In [ ]:
#create time-sensitive split for cross-validation
ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=12960,
    max_train_size=26300,
    test_size=18347,
)

#apply split to the data
all_splits = list(ts_cv.split(X, y))

#visualize train and test sets
#...?

In [ ]:
#create pipeline to run the three models, perform hyperparameter tuning and select the best performing model

def evaluate_models(X, y, cv, models):
    best_model = None
    best_score = float('-inf')

    for model in models:
        if isinstance(model, XGBRegressor):
            # Parameter grid for XGBoost model
            param_grid = {
                'learning_rate': [0.01, 0.05, 0.1],
                'n_estimators': [100, 200, 300],
                'max_depth': [3, 4, 5]
            }
        elif isinstance(model, HistGradientBoostingRegressor):
            # Parameter grid for HistGradientBoostingRegressor model
            param_grid = {
                'learning_rate': [0.01, 0.05, 0.1],
                'max_iter': [100, 200, 300],
                'max_depth': [3, 4, 5]
            }
        else:
            # Use default hyperparameters for other models
            best_model = model
            print(f"Using default hyperparameters for model: {type(model).__name__}")
            continue

        # Perform grid search with cross-validation
        grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv)
        grid_search.fit(X, y)

        # Get the best parameters and score from the search
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_
        print(f"Results of Grid Search for model: {type(model).__name__}")
        print(f"Best Parameters: {best_params}")
        print(f"Best Score: {best_score}")

        if best_score > best_model_score:
            best_model = model.set_params(**best_params)

    if best_model is not None:
        # Train the best model with the entire dataset
        best_model.fit(X, y)
        y_pred = best_model.predict(X)

        mae = mean_absolute_error(y, y_pred)
        rmse = mean_squared_error(y, y_pred, squared=False)

        print(f"Best Model: {type(best_model).__name__}")
        print(f"Mean Absolute Error:     {mae:.3f}")
        print(f"Root Mean Squared Error: {rmse:.3f}")

    return best_model


# Define the models to evaluate
models = [XGBRegressor(), HistGradientBoostingRegressor(), other_model()]

# Example usage with multiple models
best_model = evaluate_models(X, y, cv, models)


### Visualization of results of final model

In [ ]:
# Shapley values
# # replace i with index of last fold (i think tis is 4)
X_train=X.iloc[all_splits[2][0]]
X_test=X.iloc[all_splits[2][1]]
y_train=y.iloc[all_splits[2][0]]
y_test=y.iloc[all_splits[2][1]]
#use selected model
model=best_model
fit = model.fit(X_train, y_train)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

#summary plot
shap.summary_plot(shap_values, X_test)
#bar chart
shap.summary_plot(shap_values, X_test, plot_type="bar")
#explain a single prediction: see https://www.kaggle.com/code/dansbecker/shap-values
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:])
# explain more predictions: don't really know how this works
shap.force_plot(explainer.expected_value, shap_values[:500,:], X_test.iloc[:500,:])
